In [10]:
import json

from kafka import KafkaConsumer

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [11]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Andrea',
    last_name='Fox'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Andrea',
 'last_name': 'Fox',
 'client_id': 'FoxAndrea',
 'topic_prefix': 'FoxAndrea'}

Create a consumer without subscribing to any particular topic

In [12]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [13]:
general_consumer.topics()

{'AdamCurry-accelerations',
 'AdamCurry-locations',
 'AtmadjaRio-accelerations',
 'AtmadjaRio-locations',
 'ByardMegan-accelerations',
 'ByardMegan-locations',
 'CurryAdam-accelerations',
 'CurryAdam-locations',
 'DoeJohn-accelerations',
 'DoeJohn-locations',
 'DoeJohn-simple',
 'DoeagerJohn-accelerations',
 'DoeagerJohn-locations',
 'EagerBrian-accelerations',
 'EagerBrian-locations',
 'FoxAndrea-accelerations',
 'FoxAndrea-locations',
 'LoydSam-accelerations',
 'LoydSam-locations',
 'MalenchikAli-locations',
 'OkeOsmond-accelerations',
 'OkeOsmond-locations',
 'PayneBarbara-accelerations',
 'PayneBarbara-locations',
 'SahaDisha-accelerations',
 'SahaDisha-locations',
 'SearsSamuel-accelerations',
 'SearsSamuel-locations',
 'SuryaBhushan-accelerations',
 'SuryaBhushan-locations'}

Close the consumer, waiting indefinitely for any needed cleanup.

In [14]:
general_consumer.close()

In [15]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [16]:
consumer.subscription()

{'FoxAndrea-accelerations', 'FoxAndrea-locations'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [17]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Message metadata:
  Topic: FoxAndrea-locations
  Partition: 0
  Offset: 0
Message Key: 8b495e5ea8d44f6db142090b92c35c19
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: FoxAndrea-locations
  Partition: 0
  Offset: 1
Message Key: a7b90f4e9c60485abad03fd3cd0a3611
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

STOPPING MESSAGE CONSUMER


Close the consumer, waiting indefinitely for any needed cleanup.

In [19]:
consumer.close()